In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import utility script

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
test.shape

In [ ]:
data_types = train.dtypes
for n, val in enumerate(data_types):
    if val != 'int64' and val !='float64':
        print(data_types.index[n],val)

In [ ]:
cor = train.corr()
cor=cor['SalePrice'].sort_values(ascending =False)
cor.head(20)

In [ ]:
#number of missing values
def find_null(train):
    null_values = []

    for n, val in enumerate(train.isnull().sum()):
        if val != 0:
            column = train.isnull().sum().index[n]
            null_values.append([column,val])

    null_values = pd.DataFrame(null_values, columns = ['Feature', 'NumofNulls'])
 
    return null_values

In [ ]:
find_null(train)

In [ ]:
train.Fence.unique()

In [ ]:
train.GarageQual.unique()

In [ ]:
train.GarageQual.fillna('NoGarQual', inplace = True)

In [ ]:
train.GarageQual.unique()

In [ ]:
nulls = find_null(train)
nulls

In [ ]:
train.LotFrontage.unique()

In [ ]:
train.LotFrontage.dtypes

In [ ]:
def replace_nulls(dataset, column_name):
    """replace the null values in the columns with more appropriate values that won't be misintepreted """
    
    if dataset[column_name].dtypes == 'object':
        name_replacement = str(0)
        dataset[column_name].fillna(name_replacement, inplace = True)
    elif (dataset[column_name].dtypes == 'int64') or (dataset[column_name].dtypes =='float64'):
        dataset[column_name].fillna(0, inplace = True)
    
    return dataset[column_name].unique()

In [ ]:
from sklearn import preprocessing


Instead of rewriting this 16 times to labelencode the values in the columns we will just write function that does it all for us and outputs a dictionary of dictionaries for the columns****

In [ ]:
def encode_values(dataset, column_name):
     
    """
    Encodes the words in the column with a integer value
    Dataset should be a dataframe
    Column_name should be a string
    """
    #the if statement will prevent us from needlessly encoding numeric columns
    if dataset[column_name].dtypes != 'object':
        return print('This data set is not type \'object\'')
    else:
        #initialize the encoder 
        le = preprocessing.LabelEncoder()
        le.fit(dataset[column_name])
        dataset[column_name] = le.fit_transform(dataset[column_name])
        names_mapped = dict(zip(le.classes_, le.transform(le.classes_)))


        return names_mapped

In [ ]:
train['Alley'].unique()

In [ ]:
#convert nulls to 0 and labelencoder map none numerical data
nulls_features = list(nulls.Feature)


for n in range(len(nulls.Feature)):
    c = replace_nulls(train,nulls_features[n])
    
name_mapping =dict()
for n in range(len(nulls_features)):
    name_mapping[nulls_features[n]] = encode_values(train, nulls_features[n])
    

In [ ]:
train.head()

In [ ]:
train.head()

In [ ]:
objects = []
for n , val in enumerate(train.dtypes):
    if val == 'object':
        objects.append(train.dtypes.index[n])

objects  

In [ ]:

for n in range(len(objects)):
    name_mapping[objects[n]] = encode_values(train, objects[n])

In [ ]:
name_mapping

In [ ]:
data_cor = train.corr()
sales_cor=data_cor['SalePrice'].sort_values(ascending =False)
sales_cor.head(10)

In [ ]:
#let's look at the features that are negatively correlated as well
sales_cor.tail(10)

In [ ]:
#import seaborn
import seaborn as sns

ax = sns.distplot(train['SalePrice'])
ax.set_xlabel('Sales Price')
ax.set_ylabel('Sales Price')

In [ ]:
ax = sns.distplot(train['OverallQual'])
ax.set_xlabel('OverallQual')
ax.set_ylabel('OverallQual')

In [ ]:
train['OverallQual'].head()

In [ ]:
ax = sns.boxplot(x = train['OverallQual'], y= train['SalePrice'])

In [ ]:
test_set = train['OverallQual'].apply(lambda x: 1 if x < 5 else (2 if (x>=5 and x<8) else 3))
test_set 
train['Overall'] = test_set
ax = sns.boxplot(x = train['Overall'], y= train['SalePrice'])

In [ ]:
ax = sns.distplot(train['GarageArea'])

In [ ]:
# Create the default pairplot
sns.pairplot(train[['SalePrice','OverallQual','ExterQual']])

In [ ]:
name_mapping['ExterQual']

In [ ]:
#Selecting highly correlated features
top_features = data_cor.loc[sales_cor>=-0.4,:].index
top_features = list(top_features)
top_features

In [ ]:
print('Number of top features that are positively correlated: %d' % len(top_features))

In [ ]:
top_cor = data_cor.loc[sales_cor[sales_cor>=0].index,sales_cor[sales_cor>=0.3].index]

#Correlation visual for top features
def cor_plot(top_cor):
    plt.figure(figsize=(12,12))
    sns.heatmap(top_cor, annot=True, cmap=plt.cm.Reds)
    plt.show()

cor_plot(top_cor)

In [ ]:
# Create the default pairplot
sns.pairplot(train[['SalePrice','GrLivArea','TotRmsAbvGrd']])
plt.show()

In [ ]:
# Create the default pairplot
sns.pairplot(train[['SalePrice','GarageCars', 'GarageArea']])
plt.show()

In [ ]:
# Create the default pairplot
sns.pairplot(train[['SalePrice','TotalBsmtSF', '1stFlrSF']])
plt.show()

In [ ]:
#Let's drop 'TotRmsAbvGrd' from top_features

In [ ]:
top_features.remove('GarageArea')
top_features.remove('GarageYrBlt')


In [ ]:
top_features.remove('TotalBsmtSF')


In [ ]:
data_train = train[top_features]
data_train.head()

In [ ]:
# replot heatmap with top 10 features correlation
cor_plot(data_cor.loc[top_features,top_features])

In [ ]:
labels = data_train['SalePrice']
train_tp_features = data_train.drop(['SalePrice'], axis = 1)

In [ ]:
labels.head()

In [ ]:
mapping = dict()
for n, val in enumerate(top_features):
    if val in name_mapping:
        mapping[val] = name_mapping[val]
    else:
        print("{} not in name_mapping".format(val))

In [ ]:
mapping

In [ ]:
#random forest attempt 2 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

minmax_scale = preprocessing.StandardScaler()

x_scaled = minmax_scale.fit_transform(train_tp_features)


log_labels = np.log1p(labels)

In [ ]:
ax = sns.distplot(log_labels)
ax.set_xlabel('Sales Price')
ax.set_ylabel('Sales Price')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_tp_features, log_labels, test_size=0.1, random_state = 0)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics

RF = RandomForestRegressor(n_estimators=30, max_depth = 8,criterion = 'mse', max_features = 'sqrt', 
                           random_state =1 )
RF.fit(x_train,y_train)
y_pred_rf = RF.predict(x_test)

In [ ]:
plt.scatter(y_pred_rf, y_test, alpha=.7,
            color='b') #alpha helps to show overlapping data
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Random Forest Model 1')
plt.show()

In [ ]:
RF.score(x_train, y_train)

In [ ]:
metrics.r2_score(y_test, y_pred_rf)

In [ ]:
from sklearn.model_selection import GridSearchCV
RF_gs = RandomForestRegressor()
params = {'n_estimators': [50,150,300],
         'max_depth': [10, None],
         'criterion' : ['mse','mae'],
         'max_features': ['sqrt','log2',None]}
gs = GridSearchCV(RF_gs, params, cv=3, n_jobs = -1,verbose = 2)
gs_fit = gs.fit(x_scaled, log_labels)
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [ ]:
import xgboost as xgb

data_dmatrix = xgb.DMatrix(data = train_tp_features, label = log_labels)

In [ ]:
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror',max_depth = 2,gamma = 0 , learning_rate = 0.2)

In [ ]:
xg_reg.fit(x_train, y_train)
y_pred_xg = xg_reg.predict(x_test)

In [ ]:
metrics.r2_score(y_train, xg_reg.predict(x_train))

In [ ]:
metrics.r2_score(y_test, y_pred_xg)

In [ ]:
xgb_gs = xgb.XGBRegressor()
params = {'objective': ['reg:squarederror'],
         'max_depth': [2,4,6,10],
          'gamma':[0,5,10],
         'learning_rate':[0.001,0.1,0.2,0.025,0.03]}
gs2 = GridSearchCV(xgb_gs, params, cv=3, n_jobs = -1,verbose = 2)
gs2_fit = gs2.fit(train_tp_features, log_labels)
pd.DataFrame(gs2_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [ ]:
plt.scatter(y_pred_xg, y_test, alpha=.7,
            color='b') #alpha helps to show overlapping data
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('XGB')
plt.show()

In [ ]:
test.head()

In [ ]:
test_features = test[top_features[:-1]]

In [ ]:
test_nulls = find_null(test_features)

In [ ]:
test_nulls_features = list(test_nulls.Feature)


for n in range(len(test_nulls.Feature)):
    c = replace_nulls(test_features,test_nulls_features[n])
    
test_mapping =dict()
for n in range(len(test_nulls_features)):
    test_mapping[test_nulls_features[n]] = encode_values(test_features, test_nulls_features[n])

In [ ]:
test_objects = []
for n , val in enumerate(test_features.dtypes):
    if val == 'object':
        test_objects.append(test_features.dtypes.index[n])

test_objects  

In [ ]:
for n in range(len(test_objects)):
     test_mapping[test_objects[n]] = encode_values(test_features, test_objects[n])

In [ ]:
test_features.head()

In [ ]:
test_scaled = minmax_scale.fit_transform(test_features)


In [ ]:
#use RF 1 model for prediction
y_test_pred = np.expm1(RF.predict(test_features))

In [ ]:
y_test_pred_xg = np.expm1(xg_reg.predict(test_features))

In [ ]:
y_test_pred[:10]


In [ ]:
final_submission_rf1 = pd.DataFrame({"Id":test.Id,"SalePrice": y_test_pred})
final_submission_rf1.head(11)

In [ ]:
final_submission_xg = pd.DataFrame({"Id":test.Id,"SalePrice": y_test_pred_xg})
final_submission_xg.head(11)

In [ ]:
pd.DataFrame(final_submission_rf1).to_csv("housing_prices_RF1_submission_20191026.csv",index=False)


In [ ]:
pd.DataFrame(final_submission_xg).to_csv("housing_prices_XGB_submission_20191026.csv",index=False)